
---

# **Chapter 37: Advanced State Patterns**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement MVVM (Model-View-ViewModel) architecture for separation of concerns
- Apply MVI (Model-View-Intent) pattern with unidirectional data flow
- Structure applications using Clean Architecture principles (Domain, Data, Presentation layers)
- Configure Dependency Injection using get_it, injectable, and kiwi
- Organize code into feature-based modular architecture
- Manage dependencies between layers using abstraction and interfaces

---

## **Prerequisites**

- Completed Chapters 12-15: State Management (Provider, Riverpod, BLoC)
- Understanding of abstract classes and interfaces (Chapter 5)
- Familiarity with repository pattern and data layers (Chapter 28)
- Knowledge of asynchronous programming (Chapter 6)

---

## **37.1 MVVM (Model-View-ViewModel)**

MVVM separates UI logic from business logic, making code testable and maintainable. The ViewModel exposes observable state that Views react to.

### **MVVM Architecture Implementation**

```dart
// File: lib/domain/entities/user.dart
// Layer: Domain - Pure business objects, no framework dependencies

class User {
  final String id;
  final String email;
  final String name;
  final bool isVerified;

  const User({
    required this.id,
    required this.email,
    required this.name,
    required this.isVerified,
  });

  // Immutable copy method
  User copyWith({
    String? id,
    String? email,
    String? name,
    bool? isVerified,
  }) {
    return User(
      id: id ?? this.id,
      email: email ?? this.email,
      name: name ?? this.name,
      isVerified: isVerified ?? this.isVerified,
    );
  }

  @override
  bool operator ==(Object other) =>
      identical(this, other) ||
      other is User &&
          runtimeType == other.runtimeType &&
          id == other.id &&
          email == other.email;

  @override
  int get hashCode => id.hashCode ^ email.hashCode;
}

// File: lib/domain/repositories/auth_repository.dart
// Layer: Domain - Abstract repository contract

abstract class AuthRepository {
  // AsyncEither represents Result type (success or failure)
  // Using Either from dartz package or custom Result class
  Future<Result<User>> login(String email, String password);
  Future<Result<void>> logout();
  Future<Result<User>> getCurrentUser();
  Stream<User?> get authStateChanges;
}

// Result class for explicit error handling
class Result<T> {
  final T? data;
  final Exception? error;
  final bool isSuccess;

  Result._(this.data, this.error, this.isSuccess);

  factory Result.success(T data) => Result._(data, null, true);
  factory Result.failure(Exception error) => Result._(null, error, false);

  R when<R>({
    required R Function(T data) success,
    required R Function(Exception error) failure,
  }) {
    if (isSuccess && data != null) {
      return success(data);
    } else {
      return failure(error!);
    }
  }
}

// File: lib/presentation/viewmodels/login_viewmodel.dart
// Layer: Presentation - Business logic holder, framework-agnostic except ChangeNotifier

import 'package:flutter/foundation.dart';

class LoginViewModel extends ChangeNotifier {
  // Dependencies injected via constructor
  final AuthRepository _authRepository;

  LoginViewModel({required AuthRepository authRepository})
      : _authRepository = authRepository;

  // Private mutable state
  bool _isLoading = false;
  String? _errorMessage;
  User? _user;

  // Public immutable getters - View observes these
  bool get isLoading => _isLoading;
  String? get errorMessage => _errorMessage;
  User? get user => _user;
  bool get isLoggedIn => _user != null;

  // Public methods called by View (Intents)
  Future<void> login(String email, String password) async {
    // Validate inputs before calling repository
    if (email.isEmpty || password.isEmpty) {
      _setError('Email and password are required');
      return;
    }

    _setLoading(true);
    _clearError();

    try {
      final result = await _authRepository.login(email, password);

      result.when(
        success: (user) {
          _user = user;
          _setLoading(false);
        },
        failure: (error) {
          _setError(_mapErrorToMessage(error));
          _setLoading(false);
        },
      );
    } catch (e) {
      _setError('An unexpected error occurred');
      _setLoading(false);
    }
  }

  Future<void> checkAuthStatus() async {
    _setLoading(true);
    
    final result = await _authRepository.getCurrentUser();
    
    result.when(
      success: (user) {
        _user = user;
        _setLoading(false);
      },
      failure: (_) {
        _user = null;
        _setLoading(false);
      },
    );
  }

  void clearError() {
    _clearError();
  }

  // Private state mutation methods - encapsulate notifyListeners
  void _setLoading(bool value) {
    _isLoading = value;
    notifyListeners(); // Notify View to rebuild
  }

  void _setError(String message) {
    _errorMessage = message;
    notifyListeners();
  }

  void _clearError() {
    _errorMessage = null;
    notifyListeners();
  }

  String _mapErrorToMessage(Exception error) {
    // Map domain exceptions to user-friendly messages
    if (error is InvalidCredentialsException) {
      return 'Invalid email or password';
    } else if (error is NetworkException) {
      return 'Please check your internet connection';
    }
    return 'Something went wrong. Please try again.';
  }
}

// File: lib/presentation/views/login_view.dart
// Layer: Presentation - UI layer, only depends on ViewModel

class LoginView extends StatefulWidget {
  @override
  _LoginViewState createState() => _LoginViewState();
}

class _LoginViewState extends State<LoginView> {
  final _emailController = TextEditingController();
  final _passwordController = TextEditingController();

  @override
  void dispose() {
    _emailController.dispose();
    _passwordController.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    // View observes ViewModel through Provider (or any state management)
    return ChangeNotifierProvider(
      create: (_) => getIt<LoginViewModel>(), // DI container
      child: Scaffold(
        appBar: AppBar(title: Text('Login')),
        body: Consumer<LoginViewModel>(
          builder: (context, viewModel, child) {
            // View reacts to ViewModel state changes
            if (viewModel.isLoggedIn) {
              // Navigation should be handled via router, not here
              // This is simplified for demonstration
              return Center(child: Text('Welcome ${viewModel.user?.name}'));
            }

            return Padding(
              padding: EdgeInsets.all(16.0),
              child: Column(
                children: [
                  if (viewModel.errorMessage != null)
                    _ErrorBanner(
                      message: viewModel.errorMessage!,
                      onDismiss: viewModel.clearError,
                    ),
                  TextField(
                    controller: _emailController,
                    decoration: InputDecoration(labelText: 'Email'),
                    keyboardType: TextInputType.emailAddress,
                  ),
                  SizedBox(height: 16),
                  TextField(
                    controller: _passwordController,
                    decoration: InputDecoration(labelText: 'Password'),
                    obscureText: true,
                  ),
                  SizedBox(height: 24),
                  ElevatedButton(
                    onPressed: viewModel.isLoading
                        ? null
                        : () => viewModel.login(
                              _emailController.text,
                              _passwordController.text,
                            ),
                    child: viewModel.isLoading
                        ? CircularProgressIndicator()
                        : Text('Login'),
                  ),
                ],
              ),
            );
          },
        ),
      ),
    );
  }
}

class _ErrorBanner extends StatelessWidget {
  final String message;
  final VoidCallback onDismiss;

  const _ErrorBanner({required this.message, required this.onDismiss});

  @override
  Widget build(BuildContext context) {
    return Container(
      color: Colors.red[100],
      padding: EdgeInsets.all(12),
      margin: EdgeInsets.only(bottom: 16),
      child: Row(
        children: [
          Icon(Icons.error, color: Colors.red),
          SizedBox(width: 8),
          Expanded(child: Text(message)),
          IconButton(
            icon: Icon(Icons.close),
            onPressed: onDismiss,
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **Model (User)**: Immutable data class representing business entities. Lives in Domain layer, knows nothing about Flutter or UI. Uses `copyWith` for updates, overrides `==` and `hashCode` for value equality.
- **ViewModel**: Holds mutable state (`_isLoading`, `_errorMessage`) and exposes immutable getters. Contains business logic (validation, error mapping) but no UI code. Uses `ChangeNotifier` to notify Views of changes.
- **View**: Only observes ViewModel, contains zero business logic. Reacts to state changes via `Consumer` or `Selector`. User inputs (button taps) call ViewModel methods.
- **Repository Abstraction**: ViewModel depends on `AuthRepository` interface, not concrete implementation. Enables testing with mocks and swapping implementations (local vs remote).
- **Result Type**: Explicit error handling using `Result<T>` instead of exceptions for expected failures (network, validation). Makes error paths visible in the type system.

---

## **37.2 MVI (Model-View-Intent)**

MVI enforces unidirectional data flow: User generates Intents → Processor updates Model → View renders Model. State is immutable and predictable.

### **MVI Pattern Implementation**

```dart
// File: lib/presentation/mvi/counter/counter_contract.dart
// Define all states, intents, and effects

// Immutable State representing the UI at any moment
@immutable
class CounterState {
  final int count;
  final bool isLoading;
  final String? error;

  const CounterState({
    required this.count,
    this.isLoading = false,
    this.error,
  });

  // Factory constructors for common states
  factory CounterState.initial() => CounterState(count: 0);
  factory CounterState.loading(int currentCount) => CounterState(
        count: currentCount,
        isLoading: true,
      );
  factory CounterState.success(int count) => CounterState(count: count);
  factory CounterState.error(int currentCount, String message) => CounterState(
        count: currentCount,
        error: message,
      );

  CounterState copyWith({
    int? count,
    bool? isLoading,
    String? error,
  }) {
    return CounterState(
      count: count ?? this.count,
      isLoading: isLoading ?? this.isLoading,
      error: error, // Allow null to clear error
    );
  }

  @override
  bool operator ==(Object other) =>
      identical(this, other) ||
      other is CounterState &&
          runtimeType == other.runtimeType &&
          count == other.count &&
          isLoading == other.isLoading &&
          error == other.error;

  @override
  int get hashCode => count.hashCode ^ isLoading.hashCode ^ error.hashCode;
}

// User Intents (Actions)
abstract class CounterIntent {}

class IncrementIntent extends CounterIntent {
  final int amount;
  IncrementIntent([this.amount = 1]);
}

class DecrementIntent extends CounterIntent {
  final int amount;
  DecrementIntent([this.amount = 1]);
}

class ResetIntent extends CounterIntent {}

// Side Effects (one-time events like navigation, toasts)
abstract class CounterEffect {}

class ShowToastEffect extends CounterEffect {
  final String message;
  ShowToastEffect(this.message);
}

class NavigateToDetailsEffect extends CounterEffect {
  final int count;
  NavigateToDetailsEffect(this.count);
}

// File: lib/presentation/mvi/counter/counter_processor.dart
// Business logic processor

class CounterProcessor {
  final CounterRepository _repository;

  CounterProcessor(this._repository);

  // Process intent and return new state + optional effects
  Future<ProcessingResult> process(
    CounterIntent intent,
    CounterState currentState,
  ) async {
    if (intent is IncrementIntent) {
      return _handleIncrement(intent, currentState);
    } else if (intent is DecrementIntent) {
      return _handleDecrement(intent, currentState);
    } else if (intent is ResetIntent) {
      return _handleReset(currentState);
    }

    return ProcessingResult(currentState, null);
  }

  Future<ProcessingResult> _handleIncrement(
    IncrementIntent intent,
    CounterState state,
  ) async {
    // Optimistic update or loading state
    final loadingState = state.copyWith(isLoading: true, error: null);
    
    try {
      final newCount = await _repository.saveCount(state.count + intent.amount);
      return ProcessingResult(
        CounterState.success(newCount),
        ShowToastEffect('Incremented!'),
      );
    } catch (e) {
      return ProcessingResult(
        CounterState.error(state.count, 'Failed to increment'),
        null,
      );
    }
  }

  Future<ProcessingResult> _handleDecrement(
    DecrementIntent intent,
    CounterState state,
  ) async {
    if (state.count - intent.amount < 0) {
      return ProcessingResult(
        state.copyWith(error: 'Cannot go below zero'),
        ShowToastEffect('Minimum value is 0'),
      );
    }

    final newCount = state.count - intent.amount;
    return ProcessingResult(
      CounterState.success(newCount),
      null,
    );
  }

  ProcessingResult _handleReset(CounterState state) {
    return ProcessingResult(
      CounterState.initial(),
      NavigateToDetailsEffect(0),
    );
  }
}

class ProcessingResult {
  final CounterState state;
  final CounterEffect? effect;

  ProcessingResult(this.state, this.effect);
}

// File: lib/presentation/mvi/counter/counter_store.dart
// State holder and intent dispatcher

class CounterStore extends ChangeNotifier {
  final CounterProcessor _processor;
  
  CounterState _state = CounterState.initial();
  CounterState get state => _state;

  CounterStore(this._processor);

  Future<void> dispatch(CounterIntent intent) async {
    final result = await _processor.process(intent, _state);
    
    // Update state if changed
    if (result.state != _state) {
      _state = result.state;
      notifyListeners();
    }

    // Handle side effects
    if (result.effect != null) {
      _handleEffect(result.effect!);
    }
  }

  void _handleEffect(CounterEffect effect) {
    // Effects should be consumed by View or middleware
    // This is simplified - typically you'd use a Stream or callback
    if (effect is ShowToastEffect) {
      // View listens to effects via separate stream
    }
  }
}

// File: lib/presentation/mvi/counter/counter_view.dart

class CounterMVIView extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ChangeNotifierProvider(
      create: (_) => CounterStore(getIt<CounterProcessor>()),
      child: Scaffold(
        appBar: AppBar(title: Text('MVI Counter')),
        body: _CounterBody(),
        floatingActionButton: _CounterActions(),
      ),
    );
  }
}

class _CounterBody extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    // View only renders based on state
    return Center(
      child: Consumer<CounterStore>(
        builder: (context, store, child) {
          final state = store.state;
          
          return Column(
            mainAxisAlignment: MainAxisAlignment.center,
            children: [
              if (state.isLoading) CircularProgressIndicator(),
              if (state.error != null)
                Text(state.error!, style: TextStyle(color: Colors.red)),
              Text(
                '${state.count}',
                style: Theme.of(context).textTheme.headlineLarge,
              ),
            ],
          );
        },
      ),
    );
  }
}

class _CounterActions extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    final store = context.read<CounterStore>();
    
    return Column(
      mainAxisSize: MainAxisSize.min,
      children: [
        FloatingActionButton(
          onPressed: () => store.dispatch(IncrementIntent()),
          child: Icon(Icons.add),
        ),
        SizedBox(height: 8),
        FloatingActionButton(
          onPressed: () => store.dispatch(DecrementIntent()),
          child: Icon(Icons.remove),
        ),
        SizedBox(height: 8),
        FloatingActionButton(
          onPressed: () => store.dispatch(ResetIntent()),
          child: Icon(Icons.refresh),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **Unidirectional Flow**: View → Intent → Processor → State → View. Data flows one way, making state changes predictable and debuggable.
- **Immutable State**: `CounterState` is immutable. Every state change creates a new object. This enables time-travel debugging and prevents accidental mutations.
- **Intent Pattern**: User actions are encapsulated as Intent objects (IncrementIntent, DecrementIntent) rather than direct method calls. This decouples View from business logic and enables logging/tracking.
- **Effects**: One-time events (toasts, navigation) that shouldn't trigger rebuilds are separated from State via Effects. State represents UI structure; Effects represent transient actions.
- **Processor**: Pure function (Intent, State) → (State, Effect). Easy to unit test—pass intent and state, assert on result state and effects.

---

## **37.3 Clean Architecture**

Clean Architecture separates code into layers with strict dependency rules: Domain (inner) knows nothing about Data or Presentation (outer).

### **Layered Architecture Implementation**

```dart
// Layer 1: DOMAIN (Innermost - No external dependencies)
// File: lib/domain/entities/todo.dart

class Todo {
  final String id;
  final String title;
  final String description;
  final bool isCompleted;
  final DateTime createdAt;

  const Todo({
    required this.id,
    required this.title,
    required this.description,
    this.isCompleted = false,
    required this.createdAt,
  });

  Todo toggleComplete() => Todo(
        id: id,
        title: title,
        description: description,
        isCompleted: !isCompleted,
        createdAt: createdAt,
      );
}

// File: lib/domain/repositories/todo_repository.dart
// Abstract contract - Domain defines what it needs

abstract class TodoRepository {
  Future<List<Todo>> getTodos();
  Future<Todo> getTodoById(String id);
  Future<void> saveTodo(Todo todo);
  Future<void> deleteTodo(String id);
  Stream<List<Todo>> watchTodos();
}

// File: lib/domain/usecases/get_todos.dart
// Use cases encapsulate business logic

class GetTodosUseCase {
  final TodoRepository _repository;

  GetTodosUseCase(this._repository);

  Future<List<Todo>> call() async {
    final todos = await _repository.getTodos();
    // Business logic: sort by creation date, filter deleted, etc.
    return todos..sort((a, b) => b.createdAt.compareTo(a.createdAt));
  }
}

class ToggleTodoUseCase {
  final TodoRepository _repository;

  ToggleTodoUseCase(this._repository);

  Future<void> call(Todo todo) async {
    final updated = todo.toggleComplete();
    await _repository.saveTodo(updated);
  }
}

// Layer 2: DATA (Depends on Domain)
// File: lib/data/models/todo_model.dart
// Data transfer object - knows about JSON, database rows

class TodoModel {
  final String id;
  final String title;
  final String description;
  final int isCompleted; // Database uses int (0/1)
  final String createdAt; // ISO 8601 string

  TodoModel({
    required this.id,
    required this.title,
    required this.description,
    required this.isCompleted,
    required this.createdAt,
  });

  // From JSON (API)
  factory TodoModel.fromJson(Map<String, dynamic> json) => TodoModel(
        id: json['id'],
        title: json['title'],
        description: json['description'],
        isCompleted: json['is_completed'] ? 1 : 0,
        createdAt: json['created_at'],
      );

  // To JSON
  Map<String, dynamic> toJson() => {
        'id': id,
        'title': title,
        'description': description,
        'is_completed': isCompleted == 1,
        'created_at': createdAt,
      };

  // Domain Mapper - converts between Model (Data) and Entity (Domain)
  Todo toDomain() => Todo(
        id: id,
        title: title,
        description: description,
        isCompleted: isCompleted == 1,
        createdAt: DateTime.parse(createdAt),
      );

  factory TodoModel.fromDomain(Todo todo) => TodoModel(
        id: todo.id,
        title: todo.title,
        description: todo.description,
        isCompleted: todo.isCompleted ? 1 : 0,
        createdAt: todo.createdAt.toIso8601String(),
      );
}

// File: lib/data/datasources/todo_local_datasource.dart
// Concrete implementation - knows about SQLite, SharedPreferences, etc.

abstract class TodoLocalDataSource {
  Future<List<TodoModel>> getTodos();
  Future<void> cacheTodos(List<TodoModel> todos);
  Future<TodoModel?> getTodoById(String id);
  Future<void> cacheTodo(TodoModel todo);
  Future<void> deleteTodo(String id);
}

class TodoLocalDataSourceImpl implements TodoLocalDataSource {
  final Database _database; // sqflite

  TodoLocalDataSourceImpl(this._database);

  @override
  Future<List<TodoModel>> getTodos() async {
    final maps = await _database.query('todos');
    return maps.map((map) => TodoModel(
          id: map['id'] as String,
          title: map['title'] as String,
          description: map['description'] as String,
          isCompleted: map['is_completed'] as int,
          createdAt: map['created_at'] as String,
        )).toList();
  }

  @override
  Future<void> cacheTodos(List<TodoModel> todos) async {
    final batch = _database.batch();
    for (final todo in todos) {
      batch.insert(
        'todos',
        todo.toJson(),
        conflictAlgorithm: ConflictAlgorithm.replace,
      );
    }
    await batch.commit();
  }

  @override
  Future<void> cacheTodo(TodoModel todo) async {
    await _database.insert(
      'todos',
      todo.toJson(),
      conflictAlgorithm: ConflictAlgorithm.replace,
    );
  }

  @override
  Future<TodoModel?> getTodoById(String id) async {
    final maps = await _database.query(
      'todos',
      where: 'id = ?',
      whereArgs: [id],
    );
    if (maps.isEmpty) return null;
    return TodoModel.fromJson(maps.first);
  }

  @override
  Future<void> deleteTodo(String id) async {
    await _database.delete(
      'todos',
      where: 'id = ?',
      whereArgs: [id],
    );
  }
}

// File: lib/data/repositories/todo_repository_impl.dart
// Repository coordinates between local and remote sources

class TodoRepositoryImpl implements TodoRepository {
  final TodoLocalDataSource _localDataSource;
  final TodoRemoteDataSource _remoteDataSource; // Defined similarly
  final NetworkInfo _networkInfo;

  TodoRepositoryImpl(
    this._localDataSource,
    this._remoteDataSource,
    this._networkInfo,
  );

  @override
  Future<List<Todo>> getTodos() async {
    if (await _networkInfo.isConnected) {
      try {
        // Fetch from remote
        final remoteModels = await _remoteDataSource.getTodos();
        // Cache locally
        await _localDataSource.cacheTodos(remoteModels);
        // Return domain entities
        return remoteModels.map((m) => m.toDomain()).toList();
      } on ServerException {
        // Fallback to local on error
        final localModels = await _localDataSource.getTodos();
        return localModels.map((m) => m.toDomain()).toList();
      }
    } else {
      // Offline: get from local
      final localModels = await _localDataSource.getTodos();
      return localModels.map((m) => m.toDomain()).toList();
    }
  }

  @override
  Future<void> saveTodo(Todo todo) async {
    final model = TodoModel.fromDomain(todo);
    
    // Optimistic: save locally first
    await _localDataSource.cacheTodo(model);
    
    // Then sync to remote if online
    if (await _networkInfo.isConnected) {
      await _remoteDataSource.saveTodo(model);
    }
  }

  @override
  Stream<List<Todo>> watchTodos() {
    // Return local data as stream
    // In real implementation, use DAO with Stream support
    return Stream.periodic(Duration(seconds: 1)).asyncMap((_) async {
      final models = await _localDataSource.getTodos();
      return models.map((m) => m.toDomain()).toList();
    });
  }

  @override
  Future<Todo> getTodoById(String id) async {
    final model = await _localDataSource.getTodoById(id);
    if (model == null) throw NotFoundException();
    return model.toDomain();
  }

  @override
  Future<void> deleteTodo(String id) async {
    await _localDataSource.deleteTodo(id);
    if (await _networkInfo.isConnected) {
      await _remoteDataSource.deleteTodo(id);
    }
  }
}

// Layer 3: PRESENTATION (Depends on Domain, knows nothing about Data)
// File: lib/presentation/blocs/todo_bloc.dart
// Uses Domain UseCases, not Repositories directly

class TodoBloc extends Bloc<TodoEvent, TodoState> {
  final GetTodosUseCase _getTodos;
  final ToggleTodoUseCase _toggleTodo;
  final AddTodoUseCase _addTodo; // Defined in Domain

  TodoBloc({
    required GetTodosUseCase getTodos,
    required ToggleTodoUseCase toggleTodo,
    required AddTodoUseCase addTodo,
  })  : _getTodos = getTodos,
        _toggleTodo = toggleTodo,
        _addTodo = addTodo,
        super(TodoInitial()) {
    on<LoadTodosEvent>(_onLoadTodos);
    on<ToggleTodoEvent>(_onToggleTodo);
    on<AddTodoEvent>(_onAddTodo);
  }

  Future<void> _onLoadTodos(
    LoadTodosEvent event,
    Emitter<TodoState> emit,
  ) async {
    emit(TodoLoading());
    
    final result = await _getTodos();
    
    // Handle result (using Either or try-catch)
    emit(TodoLoaded(result));
  }

  Future<void> _onToggleTodo(
    ToggleTodoEvent event,
    Emitter<TodoState> emit,
  ) async {
    await _toggleTodo(event.todo);
    add(LoadTodosEvent()); // Refresh list
  }

  Future<void> _onAddTodo(
    AddTodoEvent event,
    Emitter<TodoState> emit,
  ) async {
    await _addTodo(event.title, event.description);
    add(LoadTodosEvent());
  }
}
```

**Explanation:**

- **Dependency Rule**: Domain (inner circle) has no dependencies. Data depends on Domain (implements repository interfaces). Presentation depends on Domain (uses UseCases).
- **Domain Layer**: Contains Entities (business objects), Repository interfaces (contracts), and UseCases (business logic). Pure Dart, no Flutter, no external packages.
- **Data Layer**: Contains Models (DTOs with JSON methods), DataSources (concrete implementations), and RepositoryImpl (coordinates sources). Mappers convert between Models (Data) and Entities (Domain).
- **Presentation Layer**: Contains BLoCs/ViewModels, Widgets, and navigation. Only talks to Domain via UseCases or repository interfaces.
- **Testability**: Domain can be tested without Flutter or databases. Data sources can be mocked. Each layer is independently testable.
- **Swapability**: Can switch SQLite to Hive by implementing new DataSource without touching Domain or Presentation. Can switch REST API to GraphQL by changing RemoteDataSource.

---

## **37.4 Dependency Injection**

Dependency Injection (DI) provides dependencies from outside rather than creating them inside, enabling loose coupling and testability.

### **DI Implementation Strategies**

```dart
// File: lib/di/injection.dart
// Using get_it package - Service Locator pattern

import 'package:get_it/get_it.dart';
import 'package:injectable/injectable.dart';

// Global service locator
final getIt = GetIt.instance;

// Configure injection (called in main.dart before runApp)
@InjectableInit(
  initializerName: 'init', // default
  preferRelativeImports: true,
  asExtension: true,
)
void configureDependencies() {
  // Manual registration (without injectable code generation)
  _registerManual();
  
  // Or use generated code:
  // getIt.init();
}

void _registerManual() {
  // Register as Singleton - one instance for app lifetime
  getIt.registerSingleton<Database>(Database.instance);
  getIt.registerSingleton<NetworkInfo>(NetworkInfoImpl());
  
  // Register as LazySingleton - created on first use
  getIt.registerLazySingleton<AuthRepository>(
    () => AuthRepositoryImpl(
      getIt<AuthLocalDataSource>(),
      getIt<AuthRemoteDataSource>(),
      getIt<NetworkInfo>(),
    ),
  );
  
  // Register Factory - new instance every time
  getIt.registerFactory<LoginViewModel>(
    () => LoginViewModel(authRepository: getIt<AuthRepository>()),
  );
  
  // Register Factory with parameters
  getIt.registerFactoryParam<TodoDetailsViewModel, String, void>(
    (todoId, _) => TodoDetailsViewModel(
      todoId: todoId,
      repository: getIt<TodoRepository>(),
    ),
  );
}

// Using injectable package with code generation
// File: lib/di/modules/database_module.dart

@module
abstract class DatabaseModule {
  // Pre-resolve async dependencies
  @preResolve
  Future<Database> get database => Database.open();
  
  // Provide synchronous dependencies
  @singleton
  NetworkInfo get networkInfo => NetworkInfoImpl();
  
  @lazySingleton
  HttpClient get httpClient => HttpClient();
}

// File: lib/data/repositories/auth_repository_impl.dart
// Annotate for automatic registration

@LazySingleton(as: AuthRepository)
class AuthRepositoryImpl implements AuthRepository {
  final AuthLocalDataSource _local;
  final AuthRemoteDataSource _remote;
  final NetworkInfo _network;

  AuthRepositoryImpl(this._local, this._remote, this._network);
  
  // Implementation...
}

// File: lib/presentation/viewmodels/login_viewmodel.dart
@injectable
class LoginViewModel extends ChangeNotifier {
  final AuthRepository _repository;
  
  LoginViewModel(this._repository);
  
  // Implementation...
}

// Alternative: Using Riverpod for DI (more Flutter-idiomatic)
// File: lib/di/riverpod_providers.dart

// Provider for repository
final authRepositoryProvider = Provider<AuthRepository>((ref) {
  return AuthRepositoryImpl(
    ref.watch(authLocalDataSourceProvider),
    ref.watch(authRemoteDataSourceProvider),
    ref.watch(networkInfoProvider),
  );
});

// StateNotifierProvider for ViewModel
final loginViewModelProvider = StateNotifierProvider<LoginViewModel, LoginState>((ref) {
  return LoginViewModel(ref.watch(authRepositoryProvider));
});

// FutureProvider for async initialization
final databaseProvider = FutureProvider<Database>((ref) async {
  return await Database.open();
});

// Scoped/Overridable providers for testing
final httpClientProvider = Provider<HttpClient>((ref) => HttpClient());

// In test:
// testWidgets('...', (tester) async {
//   await tester.pumpWidget(
//     ProviderScope(
//       overrides: [
//         httpClientProvider.overrideWithValue(MockHttpClient()),
//       ],
//       child: MyApp(),
//     ),
//   );
// });

// File: lib/main.dart
void main() async {
  WidgetsFlutterBinding.ensureInitialized();
  
  // Initialize DI
  configureDependencies();
  
  // Or with Riverpod:
  // runApp(ProviderScope(child: MyApp()));
  
  runApp(MyApp());
}
```

**Explanation:**

- **get_it**: Service locator pattern. Global registry of dependencies. Access via `getIt<Type>()`. Good for vanilla Flutter or small projects.
- **Injectable**: Code generation for get_it. Annotate classes with `@Injectable`, `@Singleton`, `@LazySingleton`. Build runner generates registration code, reducing boilerplate.
- **Riverpod as DI**: Providers are inherently a DI system. `ref.watch()` automatically handles dependencies and disposal. Supports scoping and overriding for tests without global state.
- **Lifecycle management**: `registerSingleton` creates immediately, `registerLazySingleton` creates on first access, `registerFactory` creates new instance per request. Use factories for ViewModels (stateful), singletons for repositories (stateless services).
- **Testing**: With DI, tests can easily mock dependencies. Either override provider registrations or use `ProviderScope` overrides with Riverpod.

---

## **37.5 Feature-Based Modular Architecture**

Organizing code by feature rather than layer improves scalability and enables code splitting.

### **Feature Module Structure**

```dart
// Project Structure:
// lib/
//   ├── core/                     # Shared utilities, theme, constants
//   │   ├── errors/
//   │   ├── usecases/
//   │   └── utils/
//   ├── features/
//   │   ├── auth/                 # Auth feature module
//   │   │   ├── data/
//   │   │   │   ├── datasources/
//   │   │   │   ├── models/
//   │   │   │   └── repositories/
//   │   │   ├── domain/
//   │   │   │   ├── entities/
//   │   │   │   ├── repositories/
//   │   │   │   └── usecases/
//   │   │   └── presentation/
//   │   │       ├── bloc/
//   │   │       ├── pages/
//   │   │       └── widgets/
//   │   ├── todos/                # Todos feature module
//   │   │   ├── data/
//   │   │   ├── domain/
//   │   │   └── presentation/
//   │   └── profile/              # Profile feature module
//   └── main.dart

// File: lib/features/auth/presentation/pages/login_page.dart
// Feature encapsulation - Auth feature exports its public API

library auth_feature;

// Export public API
export 'domain/entities/user.dart';
export 'domain/repositories/auth_repository.dart';
export 'presentation/bloc/auth_bloc.dart';
export 'presentation/pages/login_page.dart';
export 'presentation/pages/register_page.dart';

// Keep data layer internal - don't export implementations

// File: lib/features/auth/auth_module.dart
// Feature configuration and routing

class AuthModule extends Module {
  @override
  List<Bind> get binds => [
    Bind.lazySingleton<AuthRepository>((i) => AuthRepositoryImpl(
          i<AuthLocalDataSource>(),
          i<AuthRemoteDataSource>(),
          i<NetworkInfo>(),
        )),
    Bind.factory((i) => LoginBloc(authRepository: i<AuthRepository>())),
  ];

  @override
  List<ModularRoute> get routes => [
    ChildRoute('/', child: (_, args) => LoginPage()),
    ChildRoute('/register', child: (_, args) => RegisterPage()),
  ];
}

// File: lib/app_module.dart
// Root module composing all features

class AppModule extends Module {
  @override
  List<Module> get imports => [
    CoreModule(), // Shared dependencies
  ];

  @override
  List<ModularRoute> get routes => [
    ModuleRoute('/auth', module: AuthModule()),
    ModuleRoute('/todos', module: TodosModule()),
    ModuleRoute('/profile', module: ProfileModule()),
    WildcardRoute(child: (_, args) => NotFoundPage()),
  ];
}

// File: lib/features/todos/presentation/pages/todos_page.dart
// Cross-feature communication via domain events or global state

class TodosPage extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        actions: [
          // Navigate to different feature
          IconButton(
            icon: Icon(Icons.person),
            onPressed: () {
              // Using Modular router
              Modular.to.navigate('/profile/');
              
              // Or using GoRouter
              // context.go('/profile');
            },
          ),
        ],
      ),
      body: BlocBuilder<TodosBloc, TodosState>(
        builder: (context, state) {
          // Feature-specific UI
          return ListView.builder(
            itemCount: state.todos.length,
            itemBuilder: (context, index) {
              final todo = state.todos[index];
              return TodoListItem(
                todo: todo,
                onTap: () {
                  // Deep linking into feature
                  context.go('/todos/${todo.id}');
                },
              );
            },
          );
        },
      ),
    );
  }
}

// Shared kernel - Core business logic used by multiple features
// File: lib/core/events/domain_events.dart

abstract class DomainEvent {}

class UserLoggedInEvent extends DomainEvent {
  final User user;
  UserLoggedInEvent(this.user);
}

class UserLoggedOutEvent extends DomainEvent {}

// Event bus for loose coupling between features
class EventBus {
  final _controller = StreamController<DomainEvent>.broadcast();
  
  Stream<T> on<T extends DomainEvent>() {
    return _controller.stream.where((event) => event is T).cast<T>();
  }
  
  void emit(DomainEvent event) {
    _controller.add(event);
  }
  
  void dispose() {
    _controller.close();
  }
}

// Usage: Todos feature reacts to Auth events
class TodosBloc extends Bloc<TodosEvent, TodosState> {
  TodosBloc(this._eventBus) : super(TodosInitial()) {
    // Listen to cross-feature events
    _eventBus.on<UserLoggedInEvent>().listen((event) {
      add(LoadTodosEvent());
    });
    
    _eventBus.on<UserLoggedOutEvent>().listen((_) {
      add(ClearTodosEvent());
    });
  }
}
```

**Explanation:**

- **Feature encapsulation**: Each feature (auth, todos, profile) contains its own data, domain, and presentation layers. Features don't import from each other's internal directories—only from public exports.
- **Modular routing**: Using `flutter_modular` or similar, each feature defines its own routes and dependencies. Root module composes features together.
- **Dependency isolation**: Feature A cannot accidentally use Feature B's repository because it's not exported. Enforces architectural boundaries.
- **Cross-feature communication**: Use domain events (EventBus) or global state (Riverpod) for loose coupling. Auth feature emits `UserLoggedInEvent`, Todos feature listens and loads data.
- **Scalability**: Teams can work on separate features without merge conflicts. Features can be extracted into separate packages or even Flutter plugins for code sharing.

---

## **Chapter Summary**

In this chapter, we explored architectural patterns for scalable Flutter applications:

### **Key Takeaways:**

1. **MVVM**: Separates View (UI) from ViewModel (state/logic). ViewModel exposes observable state via ChangeNotifier/Stream. View reacts to state changes. Repository abstraction enables testing.

2. **MVI**: Unidirectional data flow (Intent → Processor → State → View). Immutable State objects represent UI snapshots. Intents encapsulate user actions. Effects handle one-time events like navigation.

3. **Clean Architecture**: Domain layer (entities, use cases) is independent. Data layer implements repository interfaces with concrete data sources. Presentation layer depends only on Domain. Mappers convert between Data Models and Domain Entities.

4. **Dependency Injection**: Use get_it + injectable for service location, or Riverpod for Flutter-native DI. Register singletons for services, factories for ViewModels. Enables easy mocking for tests.

5. **Feature-Based Structure**: Organize by feature (auth/, todos/, profile/) rather than layer (data/, domain/, presentation/). Each feature is self-contained with public exports. Reduces coupling and enables code splitting.

### **Next Steps:**

Chapter 38 will cover **Plugin Development**:
- Creating custom Flutter plugins
- Federated plugins architecture
- Platform channel communication
- Publishing to pub.dev

---

**End of Chapter 37**

---

# **Next Chapter: Chapter 38 - Plugin Development**

Chapter 38 will explore creating reusable plugins, platform channel communication, and the federated plugin architecture for cross-platform development.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='36. animations_mastery.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='38. plugin_development.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
